### 本地数据读取和处理

In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import time

In [2]:
def read_img(file_path):
    start = time.time()
    file_name = os.listdir(file_path)
    x = []
    y = []
    for i in file_name:
        img = Image.open(file_path+i)
        img = img.resize((28,28)).convert('L')
        img_array = np.array(img)
        label = i.split('_')[-1].split('.')[0]
        x.append(img_array)
        y.append(label)
    x = np.array(x)
    y = np.array(y)
    y = y.astype(np.int64)
    print(f'run time is {time.time() - start}')
    return x,y

In [5]:
x_test,y_test = read_img("./mnist_image_label/mnist_train_jpg_60000/")

run time is 38.015801429748535


In [6]:
x_train,y_train = read_img("./mnist_image_label/mnist_train_jpg_60000/")

run time is 7.674140930175781


In [7]:
np.save("./x_test.npy",x_test)
np.save("./x_train.npy",x_train)

In [8]:
np.save("./y_test.npy",y_test)
np.save("./y_train.npy",y_train)

In [9]:
start = time.time()
x = np.load("./x_train.npy")
x = np.load("./y_train.npy")
print(f'run time is {time.time() - start}')
start2 = time.time()
x1 = np.load("./x_test.npy")
y1 = np.load("./y_test.npy")
print(f'run time is {time.time() - start2}')

run time is 0.031000137329101562
run time is 0.027999162673950195


# 数据增强

In [10]:
import tensorflow as tf
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.05,
    horizontal_flip=False,
    zoom_range=0.1
)

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),#扁平化 相当于reshape(num,-1)
    tf.keras.layers.Dense(200,activation="relu"),#relu,sigmoid
    tf.keras.layers.Dense(100,activation="relu"),
    tf.keras.layers.Dense(50,activation="relu"),
    tf.keras.layers.Dense(10,activation='softmax')#输出层,节点数=类别数，激活函数用softmax 概率总和为1
])

In [12]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
             metrics=['accuracy'])

checkpoint_save_path = "./checkpoint/mnist.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                save_weights_only=True,
                                                save_best_only=True)

model.fit(img_gen.flow(x_train.reshape(60000,28,28,1),y_train),batch_size=64,epochs=10,
          validation_data=(x_test,y_test),
         callbacks=[cp_callback])

model.summary()

Epoch 1/10
1875/1875 [==============================] - 9s 5ms/step - loss: 1.0905 - accuracy: 0.6668 - val_loss: 35.9355 - val_accuracy: 0.8888
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5382 - accuracy: 0.8407 - val_loss: 31.6495 - val_accuracy: 0.9156
Epoch 3/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3787 - accuracy: 0.8861 - val_loss: 26.9833 - val_accuracy: 0.9362
Epoch 4/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3027 - accuracy: 0.9097 - val_loss: 22.7761 - val_accuracy: 0.9454
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2602 - accuracy: 0.9218 - val_loss: 21.8639 - val_accuracy: 0.9498
Epoch 6/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2348 - accuracy: 0.9306 - val_loss: 17.4770 - val_accuracy: 0.9594
Epoch 7/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2121 - accuracy: 0.9369 - val_loss: 18.1336 - val_ac

### 猫狗分类

In [20]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import time
from sklearn.model_selection import train_test_split

In [21]:
def read_img(file_path):
    start = time.time()
    file_name = os.listdir(file_path)
    x = []
    y = []
    for a in file_name:
        for i in os.listdir(file_path+a):
            img = Image.open(file_path+a+"/"+i)
            img = img.resize((28,28)).convert('L')
            img_array = np.array(img)
            label = i.split('.')[0]
            if label == 'cat':
                label = [0]
            else:
                label = [1]
            x.append(img_array)
            y.append(label)
    x = np.array(x)
    y = np.array(y)
    print(f'run time is {time.time() - start}')
    return x,y

In [22]:
x,y = read_img("./catVSdog/")

run time is 3.227485179901123


In [23]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [24]:
x_train.shape

(1731, 28, 28)

### 打乱标签，数据集

In [25]:
state = np.random.get_state()
np.random.shuffle(x_train)
np.random.set_state(state)
np.random.shuffle(y_train)

In [26]:
import tensorflow as tf
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.05,
    horizontal_flip=False,
    zoom_range=0.1
)

In [75]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),#扁平化 相当于reshape(num,-1)
    tf.keras.layers.Dense(200,activation="relu"),#relu,sigmoid
    tf.keras.layers.Dense(100,activation="relu"),
    tf.keras.layers.Dense(50,activation="relu"),
    tf.keras.layers.Dense(2,activation='softmax')#输出层,节点数=类别数，激活函数用softmax 概率总和为1
])

In [76]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
             metrics=['accuracy'])

checkpoint_save_path = "./checkpoint/catVSdog.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                save_weights_only=True,
                                                save_best_only=True)

model.fit(img_gen.flow(x_train.reshape(1731,28,28,1),y_train),batch_size=64,epochs=20,
          validation_data=(x_test,y_test),
         callbacks=[cp_callback])

model.summary()

Epoch 1/20
55/55 [==============================] - 1s 8ms/step - loss: 0.6782 - accuracy: 0.0745 - val_loss: 24.1575 - val_accuracy: 0.1316
Epoch 2/20
55/55 [==============================] - 0s 7ms/step - loss: 0.6758 - accuracy: 0.1670 - val_loss: 22.9255 - val_accuracy: 0.8753
Epoch 3/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6864 - accuracy: 0.2068 - val_loss: 34.3490 - val_accuracy: 0.0000e+00
Epoch 4/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6674 - accuracy: 0.0092 - val_loss: 39.4355 - val_accuracy: 0.0046
Epoch 5/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6610 - accuracy: 0.0526 - val_loss: 77.8509 - val_accuracy: 0.0046
Epoch 6/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6654 - accuracy: 0.0300 - val_loss: 59.7902 - val_accuracy: 0.0139
Epoch 7/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6610 - accuracy: 0.0774 - val_loss: 50.0949 - val_accuracy: 0.0416
Epoch 8/2

### 2

In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense
import tensorflow as tf

In [78]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),input_shape=(28,28,1),activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))


In [79]:
model.compile(optimizer='adam', #优化器Adam
              loss='binary_crossentropy', #损失函数：交叉熵损失
              metrics=['accuracy']) #准确率：accuracy:y_/y均为数值；categorical_accuracy:y_/y都是独热码；sparse_categorical_accuracy:y_/y是数值+独热码
model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 5, 5, 64)          0         
                                                                 
 flatten_6 (Flatten)         (None, 1600)             

In [80]:
checkpoint_save_path = "./checkpoint/catVSdog.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint( #保存模型cp_callback
    filepath=checkpoint_save_path,
    save_weights_only=True, #只保存weight
    save_best_only=True #只保存最好的一次
)


In [84]:
history = model.fit(img_gen.flow(x_train.reshape(1731,28,28,1),y_train),
                 epochs=20,
                 batch_size=128,
                 validation_data=(x_test,y_test),
                 validation_freq=1,
                 callbacks=[cp_callback],
                 verbose=1)


Epoch 1/20
55/55 [==============================] - 1s 12ms/step - loss: 0.2522 - accuracy: 0.9024 - val_loss: 102.1904 - val_accuracy: 0.6582
Epoch 2/20
55/55 [==============================] - 0s 7ms/step - loss: 0.2296 - accuracy: 0.9001 - val_loss: 124.7534 - val_accuracy: 0.6097
Epoch 3/20
55/55 [==============================] - 0s 5ms/step - loss: 0.2207 - accuracy: 0.9145 - val_loss: 119.9347 - val_accuracy: 0.6120
Epoch 4/20
55/55 [==============================] - 0s 5ms/step - loss: 0.2501 - accuracy: 0.8966 - val_loss: 106.5319 - val_accuracy: 0.6397
Epoch 5/20
55/55 [==============================] - 0s 5ms/step - loss: 0.2071 - accuracy: 0.9139 - val_loss: 111.3136 - val_accuracy: 0.6259
Epoch 6/20
55/55 [==============================] - 0s 5ms/step - loss: 0.2031 - accuracy: 0.9197 - val_loss: 143.7333 - val_accuracy: 0.6097
Epoch 7/20
55/55 [==============================] - 0s 5ms/step - loss: 0.2089 - accuracy: 0.9151 - val_loss: 122.9547 - val_accuracy: 0.6189
Epoch